In [1]:
import duckdb

# puzzle input goes here
input = """
"""

In [2]:
duckdb.sql("""
    WITH input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            row_no,
            GENERATE_SUBSCRIPTS(STRING_SPLIT(row_content, ''), 1) col_no,
            UNNEST(STRING_SPLIT(row_content, '')) obj
        FROM (
            SELECT
                GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) row_no,
                UNNEST(STRING_SPLIT(input, '\n')) row_content
            FROM input
        )
    ), t_inv_wall AS (
        SELECT row_no, col_no, obj
        FROM t
        UNION ALL
        SELECT 0 row_no, col_no, '#' obj
        FROM (SELECT DISTINCT col_no FROM t)
    ), t_cubes AS (
        SELECT
            col_no,
            row_no,
            COALESCE(LEAD(row_no) OVER w, (SELECT MAX(row_no) + 1 FROM t)) next_cube_row_no
        FROM t_inv_wall
        WHERE obj = '#'
        WINDOW w AS (PARTITION BY col_no ORDER BY row_no)
    ), t_rocks_between AS (
        SELECT
            t_cubes.col_no,
            t_cubes.row_no cube_row_no,
            t.row_no
        FROM t_cubes, t
        WHERE t.obj = 'O'
        AND t_cubes.col_no = t.col_no
        AND t.row_no BETWEEN t_cubes.row_no AND t_cubes.next_cube_row_no
    ), t_north AS (
        SELECT 
            col_no,
            row_no old_row_no,
            cube_row_no + COUNT(row_no) OVER w new_row_no,
            (SELECT MAX(row_no) FROM t) - new_row_no + 1 row_weight
        FROM t_rocks_between
        WINDOW w AS (PARTITION BY col_no, cube_row_no ORDER BY row_no)
    )
    SELECT SUM(row_weight) ans
    FROM t_north
""" % input)

┌────────┐
│  ans   │
│ int128 │
├────────┤
│ 112046 │
└────────┘

In [3]:
duckdb.sql("""
    WITH RECURSIVE input AS (
        SELECT '%s' AS input
    ), t AS (
        SELECT
            row_no,
            GENERATE_SUBSCRIPTS(STRING_SPLIT(row_content, ''), 1) col_no,
            UNNEST(STRING_SPLIT(row_content, '')) obj
        FROM (
            SELECT
                GENERATE_SUBSCRIPTS(STRING_SPLIT(input, '\n'), 1) row_no,
                UNNEST(STRING_SPLIT(input, '\n')) row_content
            FROM input
        )
    ), t_wall AS (
        SELECT row_no, col_no, obj
        FROM t
        UNION ALL
        SELECT 0 row_no, col_no, '#' obj
        FROM (SELECT DISTINCT col_no FROM t)
        UNION ALL
        SELECT (SELECT MAX(row_no) FROM t) + 1 row_no, col_no, '#' obj
        FROM (SELECT DISTINCT col_no FROM t)
        UNION ALL
        SELECT row_no, 0 col_no, '#' obj
        FROM (SELECT DISTINCT row_no FROM t)
        UNION ALL
        SELECT row_no, (SELECT MAX(col_no) FROM t) + 1 col_no, '#' obj
        FROM (SELECT DISTINCT row_no FROM t)
    ), t_cubes AS (
        SELECT
            col_no,
            row_no,
            LEAD(row_no) OVER w_ns north,
            LAG(row_no) OVER w_ns south,
            LEAD(col_no) OVER w_ew east,
            LAG(col_no) OVER w_ew west,
        FROM t_wall
        WHERE obj = '#'
        WINDOW w_ns AS (PARTITION BY col_no ORDER BY row_no),
            w_ew AS (PARTITION BY row_no ORDER BY col_no)
    ), t_rocks AS (
        SELECT row_no, col_no
        FROM t
        WHERE obj = 'O'
    ), t_cycle AS (
        SELECT
            0 iter_no,
            ARRAY_AGG(t_rocks ORDER BY row_no, col_no) rocks,
            ARRAY[MD5(CAST(ARRAY_AGG(t_rocks ORDER BY row_no, col_no) AS VARCHAR))] prev_hash
        FROM t_rocks
        UNION ALL
        SELECT
            iter_no,
            rocks,
            ARRAY_APPEND(prev_hash, MD5(CAST(rocks AS VARCHAR))) prev_hash
        FROM (
            SELECT
                t_iter.iter_no + 1 iter_no,
                t_iter.prev_hash,
                ARRAY_AGG({'row_no': t_iter.row_no, 'col_no': t_iter.col_no} ORDER BY t_iter.row_no, t_iter.col_no) rocks
            FROM (
                SELECT
                    t_east.row_no,
                    t_cubes.col_no - COUNT(t_east.col_no) OVER w col_no,
                    t_east.rocks,
                    t_east.iter_no,
                    t_east.prev_hash
                FROM t_cubes, (
                    SELECT
                        t_cubes.row_no - COUNT(t_south.row_no) OVER w row_no,
                        t_south.col_no,
                        t_south.rocks,
                        t_south.iter_no,
                        t_south.prev_hash
                    FROM t_cubes, (
                        SELECT
                            t_west.row_no,
                            t_cubes.col_no + COUNT(t_west.col_no) OVER w col_no,
                            t_west.rocks,
                            t_west.iter_no,
                            t_west.prev_hash
                        FROM t_cubes, (
                            SELECT
                                t_cubes.row_no + COUNT(t_north.row_no) OVER w row_no,
                                t_north.col_no,
                                t_north.rocks,
                                t_north.iter_no,
                                t_north.prev_hash
                            FROM t_cubes, (
                                SELECT
                                    UNNEST(rocks).row_no row_no,
                                    UNNEST(rocks).col_no col_no,
                                    rocks,
                                    iter_no,
                                    prev_hash
                                FROM t_cycle
                                WHERE iter_no < 10000
                                AND NOT ARRAY_CONTAINS(prev_hash[:-2], prev_hash[-1])
                            ) t_north
                            WHERE t_cubes.col_no = t_north.col_no
                            AND t_north.row_no BETWEEN t_cubes.row_no AND t_cubes.north
                            WINDOW w AS (PARTITION BY t_cubes.col_no, t_cubes.row_no ORDER BY t_north.row_no)
                        ) t_west
                        WHERE t_cubes.row_no = t_west.row_no
                        AND t_west.col_no BETWEEN t_cubes.col_no AND t_cubes.east
                        WINDOW w AS (PARTITION BY t_cubes.row_no, t_cubes.col_no ORDER BY t_west.col_no)
                    ) t_south
                    WHERE t_cubes.col_no = t_south.col_no
                    AND t_south.row_no BETWEEN t_cubes.south AND t_cubes.row_no
                    WINDOW w AS (PARTITION BY t_cubes.col_no, t_cubes.row_no ORDER BY t_south.row_no)
                ) t_east
                WHERE t_cubes.row_no = t_east.row_no
                AND t_east.col_no BETWEEN t_cubes.west AND t_cubes.col_no
                WINDOW w AS (PARTITION BY t_cubes.row_no, t_cubes.col_no ORDER BY t_east.col_no)
            ) t_iter
            GROUP BY ALL
        )
    ), iters AS (
        SELECT MIN(iter_no) min_iter, MAX(iter_no) max_iter
        FROM t_cycle
        WHERE rocks = (
            SELECT LAST(rocks) last_rocks
            FROM t_cycle
        )
    ), last_position AS (
        SELECT (SELECT MAX(row_no) FROM t) - UNNEST(rocks).row_no + 1 row_weight
        FROM t_cycle, iters
        WHERE iter_no = ((1000000000 - min_iter) %% (max_iter - min_iter)) + min_iter
    )
    SELECT SUM(row_weight) ans
    FROM last_position
""" % input)

┌────────┐
│  ans   │
│ int128 │
├────────┤
│ 104619 │
└────────┘